# Generate camera parameters

In [24]:
import json
import numpy as np
from pathlib import Path
import pickle


In [22]:
with open(Path("AcinoSet_2017_08_29_top_jules_run1_1/dlc/cam1DLC_resnet152_CheetahOct14shuffle1_500000includingmetadata.pickle"), "rb") as f:
    dlc_data = pickle.load(f)

In [23]:
dlc_data["data"]["DLC-model-config file"]

{'stride': 8.0,
 'weigh_part_predictions': False,
 'weigh_negatives': False,
 'fg_fraction': 0.25,
 'weigh_only_present_joints': False,
 'mean_pixel': [123.68, 116.779, 103.939],
 'shuffle': True,
 'snapshot_prefix': '/data/daniel/Cheetah-UCT-2019-10-14/dlc-models/iteration-27/CheetahOct14-trainset95shuffle1/test/snapshot',
 'log_dir': 'log',
 'global_scale': 0.8,
 'location_refinement': True,
 'locref_stdev': 7.2801,
 'locref_loss_weight': 1.0,
 'locref_huber_loss': True,
 'pairwise_huber_loss': True,
 'pairwise_loss_weight': 0.1,
 'optimizer': 'sgd',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'regularize': False,
 'weight_decay': 0.0001,
 'mirror': False,
 'crop_pad': 0,
 'scoremap_dir': 'test',
 'batch_size': 1,
 'dataset_type': 'default',
 'deterministic': False,
 'crop': False,
 'cropratio': 0.25,
 'minsize': 100,
 'leftwidth': 400,
 'rightwidth': 400,
 'topheight': 400,
 'bottomheight': 400,
 'all_joints': [[0],
  [1],
  [2],
  [3],
  [4],
  [5],


In [14]:
cam1_body = dlc_data["data"]["DLC-model-config file"]["all_joints_names"]

In [16]:
cam1_body

['r_eye',
 'l_eye',
 'r_shoulder',
 'r_front_knee',
 'r_front_ankle',
 'r_front_paw',
 'spine',
 'r_hip',
 'r_back_knee',
 'r_back_ankle',
 'r_back_paw',
 'tail1',
 'tail2',
 'l_shoulder',
 'l_front_knee',
 'l_front_ankle',
 'l_front_paw',
 'l_hip',
 'l_back_knee',
 'l_back_ankle',
 'l_back_paw',
 'lure',
 'tail_base',
 'nose',
 'neck_base']

## Intrinsic params

In [75]:
# Inspect DLC camera 1 intrinsic params
with open("AcinoSet-Cam12-testDLC-2025-10-31-3d/camera_matrix/camera-1_intrinsic_params.pickle", "rb") as f:
    dlc_cam1_params = pickle.load(f)

print(dlc_cam1_params)

{'camera-1': {'mtx': array([[1.05736413e+03, 0.00000000e+00, 9.44956686e+02],
       [0.00000000e+00, 7.21560496e+02, 5.41798677e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), 'dist': array([[-4.32992622e+01,  3.06471162e+03, -2.64524199e+00,
        -1.01478504e+00, -3.84828659e+05]]), 'objpoints': [array([[0., 0., 0.],
       [1., 0., 0.],
       [2., 0., 0.],
       [3., 0., 0.],
       [4., 0., 0.],
       [5., 0., 0.],
       [6., 0., 0.],
       [7., 0., 0.],
       [0., 1., 0.],
       [1., 1., 0.],
       [2., 1., 0.],
       [3., 1., 0.],
       [4., 1., 0.],
       [5., 1., 0.],
       [6., 1., 0.],
       [7., 1., 0.],
       [0., 2., 0.],
       [1., 2., 0.],
       [2., 2., 0.],
       [3., 2., 0.],
       [4., 2., 0.],
       [5., 2., 0.],
       [6., 2., 0.],
       [7., 2., 0.],
       [0., 3., 0.],
       [1., 3., 0.],
       [2., 3., 0.],
       [3., 3., 0.],
       [4., 3., 0.],
       [5., 3., 0.],
       [6., 3., 0.],
       [7., 3., 0.],
       [

In [22]:
for key in dlc_cam1_params["camera-1"].keys():
    entity = dlc_cam1_params["camera-1"][key]
    if isinstance(entity, (list, tuple)):
        print(f"{key}: {[e.shape for e in entity]}")
    else:
        print(f"{key}: {entity.shape}")

mtx: (3, 3)
dist: (1, 5)
objpoints: [(48, 3), (48, 3)]
imgpoints: [(48, 1, 2), (48, 1, 2)]


In [ ]:
# Load AcinoSet intrinsic calibration data
intrinsic_calib = {
    "camera": Path("C:/Users/locha/Downloads/intrinsic_calib/camera.json"),
    "points": Path("C:/Users/locha/Downloads/intrinsic_calib/points.json"),
}
with open(intrinsic_calib["camera"], "r") as f:
    camera_intrinsic = json.load(f)
with open(intrinsic_calib["points"], "r") as f:
    points_intrinsic = json.load(f)


In [56]:
imgpoints = []
for val in points_intrinsic["points"].values():
    imgpoints.append(np.array(val).reshape(48, 1, 2))

In [63]:
camera1_intrinsic_params = {
    "mtx": np.array(camera_intrinsic["k"]),
    "dist": np.pad(
                np.array(camera_intrinsic["d"]).flatten(), 
                (0, 1), 
                constant_values=0
            ).reshape(1, 5),
    "objpoints": [dlc_cam1_params["camera-1"]["objpoints"][0]] * len(imgpoints),
    "imgpoints": imgpoints,
}

In [ ]:
# Overwrite and save updated params
# camera-2 intrinsic params just 
# has a copy of camera-1 params in key = "camera-1"
# In the AcinoSet case, all cameras have the same intrinsic parameters
# so we can just copy them over
dlc_cam1_params2 = dlc_cam1_params.copy()
dlc_cam1_params2["camera-1"] = camera1_intrinsic_params
dlc_cam1_params2["camera-2"] = camera1_intrinsic_params
with open("AcinoSet-Cam12-testDLC-2025-10-31-3d/camera_matrix/camera-2_intrinsic_params2.pickle", "wb") as f:
    pickle.dump(dlc_cam1_params2, f)

## Stereo params

In [80]:
# Inspect DLC stereo params
with open("AcinoSet-Cam12-testDLC-2025-10-31-3d/camera_matrix/stereo_params.pickle", "rb") as f:
    dlc_stereo_params = pickle.load(f)

print(dlc_stereo_params)

{'camera-1-camera-2': {'cameraMatrix1': array([[1.05736413e+03, 0.00000000e+00, 9.44956686e+02],
       [0.00000000e+00, 7.21560496e+02, 5.41798677e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), 'cameraMatrix2': array([[8.61143313e+02, 0.00000000e+00, 9.34672086e+02],
       [0.00000000e+00, 1.15162835e+03, 5.24692124e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), 'distCoeffs1': array([[-4.32992622e+01,  3.06471162e+03, -2.64524199e+00,
        -1.01478504e+00, -3.84828659e+05]]), 'distCoeffs2': array([[-4.22037241e+00,  7.58855801e+01, -1.64751888e-01,
         3.15647482e-01, -2.90691810e+02]]), 'R': array([[ 0.8345396 ,  0.0320563 ,  0.55001459],
       [-0.54605093, -0.08468239,  0.83346102],
       [ 0.07329423, -0.99589221, -0.05316644]]), 'T': array([[-76.91202475],
       [-82.38659476],
       [ 85.45349661]]), 'E': array([[ 40.62349932,  89.28457421, -66.84195681],
       [ 76.95153411, -73.85676325,  42.91153172],
       [110.75275814,   9

In [83]:
dlc_stereo_params["camera-1-camera-2"].keys()

dict_keys(['cameraMatrix1', 'cameraMatrix2', 'distCoeffs1', 'distCoeffs2', 'R', 'T', 'E', 'F', 'R1', 'R2', 'P1', 'P2', 'roi1', 'roi2', 'Q', 'image_shape'])

In [ ]:
# Load AcinoSet extrinsics
extrinsic_calib = {
    "camera": Path("C:/Users/locha/Downloads/extrinsic_calib/6_cam_scene_sba.json"),
    "manual_points": Path("C:/Users/locha/Downloads/extrinsic_calib/points/manual_points.json"),
    "points1": Path("C:/Users/locha/Downloads/extrinsic_calib/points/points1.json"),
    "points2": Path("C:/Users/locha/Downloads/extrinsic_calib/points/points2.json"),
}
with (
    open(extrinsic_calib["camera"], "r") as f1, 
    open(extrinsic_calib["points1"], "r") as f2,
    open(extrinsic_calib["points2"], "r") as f3,
    open(extrinsic_calib["manual_points"], "r") as f4
):
    camera_extrinsic = json.load(f1)
    points1_extrinsic = json.load(f2)
    points2_extrinsic = json.load(f3)
    manual_points_extrinsic = json.load(f4)

In [ ]:
# Convert to OpenCV format: list of (N, 1, 2) arrays
imgpoints1 = [np.array(p, dtype=np.float32).reshape(-1, 1, 2) for p in points1_extrinsic["points"].values()]
imgpoints2 = [np.array(p, dtype=np.float32).reshape(-1, 1, 2) for p in points2_extrinsic["points"].values()][:len(imgpoints1)]
objpoints_extrinsic = [dlc_cam1_params["camera-1"]["objpoints"][0]] * len(imgpoints1)

In [115]:
K1 = camera1_intrinsic_params["mtx"].copy()
K2 = camera1_intrinsic_params["mtx"].copy()
D1 = camera1_intrinsic_params["dist"].copy()
D2 = camera1_intrinsic_params["dist"].copy()
width, height = camera_extrinsic["camera_resolution"]

In [116]:
import cv2

ret, K1, D1, K2, D2, R, T, E, F = cv2.stereoCalibrate(
    objpoints_extrinsic, imgpoints1, imgpoints2,
    K1, D1, K2, D2,
    imageSize=(width, height),
    flags=cv2.CALIB_FIX_INTRINSIC
)


In [117]:
R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
    K1, D1, K2, D2,
    imageSize=(width, height),
    R=R, T=T
)


In [135]:
new_stereo_params = dlc_stereo_params.copy()
new_stereo_params["camera-1-camera-2"]["cameraMatrix1"] = K1
new_stereo_params["camera-1-camera-2"]["cameraMatrix2"] = K2
new_stereo_params["camera-1-camera-2"]["distCoeffs1"] = D1
new_stereo_params["camera-1-camera-2"]["distCoeffs2"] = D2
new_stereo_params["camera-1-camera-2"]["R"] = R
new_stereo_params["camera-1-camera-2"]["T"] = T
new_stereo_params["camera-1-camera-2"]["E"] = E
new_stereo_params["camera-1-camera-2"]["F"] = F
new_stereo_params["camera-1-camera-2"]["R1"] = R1
new_stereo_params["camera-1-camera-2"]["R2"] = R2
new_stereo_params["camera-1-camera-2"]["P1"] = P1
new_stereo_params["camera-1-camera-2"]["P2"] = P2
new_stereo_params["camera-1-camera-2"]["roi1"] = roi1
new_stereo_params["camera-1-camera-2"]["roi2"] = roi2
new_stereo_params["camera-1-camera-2"]["Q"] = Q
new_stereo_params["camera-1-camera-2"]["image_shape"] = [(width, height)] * len(imgpoints1)
with open("AcinoSet-Cam12-testDLC-2025-10-31-3d/camera_matrix/stereo_params2.pickle", "wb") as f:
    pickle.dump(new_stereo_params, f)